In [ ]:
import os
import sys
from time import sleep

import numpy as np

import GCode
import GRBL


# Draw a Brick Pattern

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

Attempt at programmatically making a brick pattern. 

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)



In [5]:
BrickHeight = 65
BrickLength = 225
BrickDepth = 12.5
BrickRatio = 215 / 65


In [6]:
BlockHeight = 89.0
BlockLength = 176.0


In [7]:
N_BrickRows = 5
H_Block_Brick = BlockHeight / N_BrickRows
L_Block_Brick = np.round(H_Block_Brick * BrickRatio, 4)


# Code:

In [1]:
default_line = np.array([[0, 0], [10, 5]])
default_feed = 300
default_power = 150

prog = GCode.GCode()


def line(
    points=default_line,
    feed=default_feed,
    power=default_power,
    dynamic_power=True,
):
    points = np.round(points, 4)

    print(points)
    print()

    prog.G0(X=points[0, 0], Y=points[0, 1])

    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M4(S=power)

    for row_idx in range(1, points.shape[0]):
        prog.G1(X=points[row_idx, 0], Y=points[row_idx, 1], F=feed)
    prog.M5()


line()
prog


NameError: name 'np' is not defined

In [38]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.round(
        np.arange(L_Block_Brick, BlockLength, L_Block_Brick), 4
    ),  # Odd rows.
    np.round(
        np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick), 4
    ),  # Even rows.
)
horizontal_brick_lines = np.round(
    np.linspace(0, BlockHeight, N_BrickRows, endpoint=False), 4
)

prog = GCode.GCode()
for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.round(
        np.array(
            [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
        ),
        4,
    )

    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]
    row_line_points = np.round(np.matmul(transform, row_line_points), 4)
    line(row_line_points)

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, 2)]
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = np.round(transform_tuple[np.mod(idx2, 2)], 4)

        column_line_points = np.round(
            np.array(
                [
                    [start_point_y, vertical_brick_line],
                    [end_point_y, vertical_brick_line],
                ]
            ),
            4,
        )
        column_line_points = np.matmul(transform, column_line_points)
        line(column_line_points)
prog


[[  0.   17.8]
 [176.   17.8]]

[[ 0.     29.4384]
 [17.8    29.4384]]

[[ 0.     88.3154]
 [17.8    88.3154]]

[[  0.     147.1922]
 [ 17.8    147.1922]]

[[  0.   35.6]
 [176.   35.6]]

[[17.8    58.8769]
 [35.6    58.8769]]

[[ 17.8    117.7538]
 [ 35.6    117.7538]]

[[  0.   53.4]
 [176.   53.4]]

[[35.6    29.4384]
 [53.4    29.4384]]

[[35.6    88.3154]
 [53.4    88.3154]]

[[ 35.6    147.1922]
 [ 53.4    147.1922]]

[[  0.   71.2]
 [176.   71.2]]

[[53.4    58.8769]
 [71.2    58.8769]]

[[ 53.4    117.7538]
 [ 71.2    117.7538]]



<GCode>[cmds=56]

In [39]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()


In [40]:
home_timeout = 360

cnc.home()
for _ in range(home_timeout):
    if len(cnc.cmd("")) == 2:
        break
    sleep(1)


In [41]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=200)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0, Z=0)
    return program


In [42]:
cnc.run(init())


0.7268331050872803

In [43]:
cnc.run(prog)


^C


11958.478502988815